## Introduction
The goal of this guide is introduce, or re-introduce you to the basics of a random forest. You won't find many formulas or formal algorithm definitions. I hope you will find a simple intuitive understanding of what a random forest is, how it works, why it works and how to build one in python. I also will provide references to further reading for those interested in digging deeper. 

We're going to start with a 30 second summary of what a random forest is, then break that down into the basic building blocks. Once we are familiar with the building blocks we will explore each of them in a little more detail. Finally we will put everything we've learned together by implementing the random forest algorithm in python.

## 30 Second summary

A random forest is made up of decision trees. A decision tree involves segementing the predictor space into simple regions. In order to make a prediction for a given observation, we take the mean (for regression) or the mode (for classification) of the training observations in the region to which it belongs. 

The set of rules used to define each region can be summarised as a tree, hence the name. A random forest grows many such trees and takes the mean or mode of predictions across the trees to achieve improved predictive performance compared to a sinlge decision tree.

If this doesn't make sense yet, don't worry about it, it will make sense as we step through it in more detail in the following sections. We are going to grow a random forest for regression today, but the pricinples we will learn apply to clasiifcation as well. The only differences are terminal node calculations and cost function selection.

## Building blocks

We've learned a random forest is made up of decision trees.  How many trees are in the forest? The number of decision trees is determined by the user. Selecting the number of decision trees is important and often comes down to a simple cost-benefit equation: the cost of calculating more trees vs the possible benefit of  increased performance. 

To create multiple decision trees from the same training data, a technique referred to as bagging is used. Bagging is a common process in machine learning and is not limited to tree based methods.  Essentially bagging is building multiple models, each based on a sample of the  training data. 

To generate these samples, bootstrapping is used. To understand bootstrapping, imagine you have a tiny data set:



In [25]:
import pandas as pd
import numpy as np

np.random.seed(0)
toy_data = pd.DataFrame({'a' : np.random.choice(57, 10), 'c' : np.random.choice(11, 10),
                         'y' : np.random.choice(78, 10)})

display(toy_data)

,a,c,y
0,44,2,46
1,47,4,37
2,53,7,25
3,0,6,77
4,3,8,72
5,3,8,9
6,39,10,20
7,9,1,69
8,19,6,47
9,21,7,64


To boostrap this tiny data, you first need to randomly select one row. The row selected is our first observation sampled from `toy_data`. Next you sample another row, noting that the row you sampled first remains in the pool of possible rows to be selected. Repeat this process until you have the number of observations you would like. Now you have a bootstrapped sample. When growing a random forest the number of rows selected through bootstrapping will generally be eqaul to the number of rows in the training data. The number of bootstrapped samples you need is equal to the number of decision trees you need to grow.

As you can see, bootstrapping is simply sampling with replacement from the training data, with the number of rows in each sample set to the number of rows in the training data and the number of samples set to the number of trees required for your forest.

So we have a number of decision trees, grown based on bootstrapped samples of our training data. Do we have a random forest yet? Not quite. We need to address the random part of the random forest. In each stage of growing a decision trees in the random forest, all predictors are considered in order to determine which is best to determine the next step in the tree. In a random forest tree, at each of these stages a random sample of possible predictors is taken. This limits which predictors can be chosen for each stage. 

Why is this important? Imagine three trees grown with this modified process, compared to three trees grown with the standard process. The three random forest trees will most likely be less similar to each other, because they have each been forced to consider a randomly selected set of predictors. Each tree is likely considering predictors other trees have ignored. Compare this with the standard process - these tree will most likely be quite similar to each other. They have all considered the same set of predictors at each stage, the only difference is the bootstrapped sample they recieve as training input. 

The result of the random forest tree process is reduced correlation among trees.  A prediction in a random forest as simply a summary of the predictions from the the decision trees in the forest. The goal of summarising over many trees is to reduce variance. Summarising over a set of less correlated trees will reduce variance more than summarising over more correlated trees. Take home? A random forest will generally have less variance than just bagging a decision tree. Put another way, the random forest predictions will be more consistent even when the test data is quite different from the training data.

Now that we have an understanding of the basic building blocks, let's explore them, one at a time.


## Decision trees
The most important component of a random forest is the decision tree. As we talked about earlier a random forest is simply a collection of decision trees with some extra rules about how the decision trees are grown. 

A decision tree is made up of nodes. Referring back to our 30 second summary, a node is a rule that helps create the regions defined by a decision tree. Let's explore this more with an example.

In [26]:
display(toy_data)

,a,c,y
0,44,2,46
1,47,4,37
2,53,7,25
3,0,6,77
4,3,8,72
5,3,8,9
6,39,10,20
7,9,1,69
8,19,6,47
9,21,7,64


We will use our toy data again. Imagine, a and c are predictors of y. To create a node, we need to select a predictor and a value to define a rule for this node. An example of such a rule would be "a < 44". A node with this rule would create two new data sets:

In [27]:
display(toy_data[toy_data['a'] < 44])
display(toy_data[toy_data['a'] >= 44])

,a,c,y
3,0,6,77
4,3,8,72
5,3,8,9
6,39,10,20
7,9,1,69
8,19,6,47
9,21,7,64


,a,c,y
0,44,2,46
1,47,4,37
2,53,7,25


And that is our root node, it splits the training data in two, based on its rule. To create a whole decision tree we simply repat this process on each of the "branches" until we reach a terminal node. 

But, how do we select the right rules for the nodes? We need some criteria. Let's think about the purpose of splitting the data. If the prediction we produce with the tree is the mean of the final split to which the obervation belongs,  we want the observations in the final split to be as similar as possible. That is, we want low variance. It follows that the criteria for selecting rules for each node needs to minimise var(y[p < s]) + var(y[p >= s]).

Where y is the outcome variable, p is a predictor and s is a the value at whcih the predictor is split.

Let's try this with our `toy_data`. First we define a function to calculate var(y[p < s]) + var(y[p >= s]).

In [28]:
def combined_RSS(y1, y2):
    
    '''calculate the combined residual sum of squares for y1 and y1'''
    
    return np.sum((y1 - np.mean(y1))**2) + np.sum((y2 - np.mean(y2))**2)

Pretty simple right? Just calculate the variance for each branch and add them together. Now we need to generate all the possible splits to assess with the critera.

In [29]:
def get_feature_splits(y, f):
   
    '''split the y variable by split s in feature f'''
    
    return ([{'feature' : f.name, 'split' :  s, 'left': y[f < s],
              'right' :  y[f >= s]} for s in f])

Here we create a list of dictionaries for each feature, where each entry contains a feature, a split and the left and right branches created by the split. Next we need to calculate the variance for each possible splits.

In [30]:
def cost_function(feature_split): 
    
    '''calculate the total RSS for the splits defined by splitting feature f at split s'''
    
    return ({'feature' : feature_split['feature'],
              'split' : feature_split['split'],
              'cost' : combined_RSS(feature_split['left'], feature_split['right'])
            }
           )

Finally we need to select the feature split with the minimum variance, and that is our first node.

In [31]:
import math
import random as rand

def flatten_list(l):
    return [item for sublist in l for item in sublist]
    
def get_split_costs(df):
    
    '''calulate the cost for each split'''
    
    return flatten_list(
        [[cost_function(split) for split in get_feature_splits(
            df.iloc[:,-1], df.iloc[:,:-1].loc[:, feature])] for feature in rand.sample(list(
            df.iloc[:,:-1].columns), int(math.sqrt(len(list(df.iloc[:,:-1].columns)))))])

def select_node(split_costs): 
       
    '''select the lowest cost node'''
    
    return (rand.choice([{'feature' : split['feature'], 'split' : split['split'],
                          'cost' : split['cost']}
                         for split in split_costs if split['cost'] == np.min(
                             [split['cost'] for split in split_costs])]))
    

There is a bit more going on here, let's break it down. The first function is just a helper to flatten lists. The second function `get_split_cost` executes `cost_function` on the all the  feature splits generated by `get_feature_splits`.  Because this is a random forest tree this function also limits the features at each split to a random sample. More on that later.

Finally, `select_node` scans all the split varainces and selects the split with the lowest variance. And we have our first node. Let's run it on `toy_data` and see what we get.


In [32]:
root_node = select_node(get_split_costs(toy_data))
display(root_node)

{'feature': 'c', 'split': 10, 'cost': 4268.222222222222}

For the `toy_data`, the root node is defined by the rule a < 53 and the combined variances of the branches is 674.75. 

Once we have the root node, we simply repeat the above process on each branch until a terminal node is reached. What makes a node terminal? We need some more criteria. 

It turns out defining criteria for a terminal node is really simple. Tree depth or node size. Tree depth referes to the number of nodes in a tree, or how many times the training data has been split. Node size is the number of observations in that node. That is, the number of observations that satisfy the rule defining the node. 

So to define the terminal node, first we set a maximum tree depth, which controls tree complexity. The more complex the tree is, the more it is influenced by the training data. This is called variance. Generally we want to keep variance low, while also managing model bias. A bias model isn't influenced enough by the training data.  So we want a balance of the two - a model that is influenced just enough by the training data. "Just enough" is tricky to define. Mostly we want to glean as much information as we can from the training data, without following it too closely, because we want the model to work for other samples from our problem data space. Next we set a minnimum node size. Because we take the mean (or mode in classification) of the terminal node, we don't want the node sizr to be so small, the mean/mode is meaningless. Also if there is only one or two observations in our terminal nodes we have probably followed the training data too closely and the model will be highly variable.

Ok, so no we now how to stop. How do we efficiently repeat the branching process? We use recursion. Recursion is just repitition, but it can be tricky because it is a *process that repeats itself*. So the logic can get a bit slippery. I'm not going to go into recursion deeply here, suffice to say its worth looking to if you want to build a decision tree. [Here](https://medium.com/@siddharthgupta555t/finally-understanding-recursion-and-binary-search-trees-857c85e72978) is a good  place to start your recursive journey.  Just to make it a little trickier, we will be using tree based recursion, because we are building a tree. The article above introduces you to general recursion and tree base recursion, so have a read of that and I will see you when you are done.

So, you have crushed recursion (or have decided you don't care), let's grow a decision tree.

A recursive process has two parts, the base case and the recursive case. The stopping criteria tells us which to apply. When the stoppig criteria is met, return the base case, if not, return the recursive case. Let's do it.

First, lets write a helper function to handle the base case, its really simple.

In [33]:
def terminal_node(node_y):
    
    '''return the prediction for the terminal node'''
    
    return np.mean(node_y)

Done. Return the mean. Now, the fun part. 

In [34]:
def grow_tree(df, node, depth, max_depth = 1, min_size = 5 ):
    
    ''' recursively grow a decision tree by applying the function to each node it 
    returns until max depth of min size criteria is met '''
    
    left = df.loc[df.loc[:, node['feature']] < node['split']]
    right = df.loc[df.loc[:, node['feature']] >= node['split']]
    
    if left.empty or right.empty:
        return terminal_node(list(left.iloc[:, -1]) + list(right.iloc[:, -1]))
        
    elif depth >= max_depth:
        return {'node': node, 
                'left': terminal_node(left.iloc[:, -1]), 
                'right': terminal_node(right.iloc[:, -1])}
    
    else:
        return {'node' : node,
                
                'left' : (lambda x: terminal_node(list(x.iloc[:, -1])) 
                          if len(x.iloc[:, -1]) <= min_size
                          else grow_tree(x, select_node(get_split_costs(x)),
                                        depth + 1, max_depth, min_size))(left),
                
                'right' : (lambda x: terminal_node(list(x.iloc[:, -1])) 
                           if len(x.iloc[:, -1]) <= min_size 
                           else grow_tree(x, select_node(get_split_costs(x)),
                                        depth + 1, max_depth, min_size))(right)
               }
    

Let's look at this function step by step. The first statements are just naming the left and right node of the first split (one of the inputs to this function is the result of the `select_node` function.  Next is our first base case. I say first because we have 3 possible stopping conditions, which gives us three scenarios in which we return terminal node, which is our base case output.

So our first stopping criteria is no split possible. We haven't discussed this yet, and it is pretty simple. When the split results in one branch rather than two. Just means  all the values of the input either satify the rule or do not satisfy it. So if that happens we return the mean of the node. Done.

Next criteria is max depth. You can see we have depth and max depth as inputs, if depth is greater or equal to max depth, return the left and right nodes as terminal nodes.

The third stopping criteria is embedded in the recursive case. This is an artifact of tree based recursion. The third criteria  applies to each of the branches created by the input node separately. We simply check the size of each node, if its less than or equal to the min size, return the terminal node for that branch.

To understand why we apply the criteria separately, think about the case where the left branch  is less than the minnimum size but the right is not.

So that is all of our base cases, on to the recursive case.

The recursive case says "If neither of the outer stopping criteria are met and the inner criteria is not met, return yourself with the depth increased by one." This is done for the left branch and the irhg branch separately. That's it. 

To understand why this approach works, let's work through one recursion. If you have read the recursion article referenced above, recall the dicussion about the call stack. That is the key to understanding why this function works. It was for me anyway.

Imagine we call `grow_tree((toy_data, root_node, 1, 2,  3 )`.  Our first  layer in the stack is the this funcion. We evaluate the stopping criteria. left or right empty is false, max depth is false. Left min size is false, so we call `grow_tree((left, select_node(get_split_costs(left)), depth + 1, max_depth, min_size)` and this goes to the next layer of our stack.  So we execute this recursed function, left or right empty is false, but max depth is true, so we return the current node, with its left and right terminal nodes. That stack is popped and we move to the next step oin our base stack, the right branch, the same thing happens, we add a stack, and pop it, then we pop our bottom layer and return our 2 depth tree. 

Lets run it and look at the result




In [35]:
grow_tree(toy_data, root_node, 1, 2, 2)

{'node': {'feature': 'c', 'split': 10, 'cost': 4268.222222222222},
 'left': {'node': {'feature': 'a', 'split': 47, 'cost': 3382.8571428571427},
  'left': 54.857142857142854,
  'right': 31.0},
 'right': 20.0}

Turns out our best first split only leaves us with one entry in the right branch, so the right branch in our example actually terminates without recursion, because it meets the minimum node size stopping criteria. And there we have it, a recursive function to grow a decision tree. We have our first building block: the decision tree, next let's bag this decision tree to give us a forest. 

## Bagging

To bag a decision tree, bootstrapping is used. As discussed earlier bootstraping is imply re-sampling from the training data with replacement. Let's do it


In [37]:
def bootstrap(df, random_state):
    return df.sample(len(df), replace = True, random_state = random_state)

You can see I've used the `pandas` `sample` method. See docs [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html/).

That's all there is to it. Now to see it in action.

In [38]:
def grow_random_forest(df, max_depth = 1, min_size = 5, no_of_trees = 10):  
    return [grow_tree(
        bootstrap(df, i), select_node( get_split_costs(bootstrap(df, i))),
        1, max_depth, min_size) for i in range(0, no_of_trees)]

To grow a forest, build trees from bootstrapped samples of the input data until you reach the desired number of trees. Done. Let's try it.

In [41]:
grow_random_forest(toy_data, 2, 3, 3)

[{'node': {'feature': 'c', 'split': 7, 'cost': 4379.6},
  'left': {'node': {'feature': 'c', 'split': 6, 'cost': 264.5},
   'left': 57.5,
   'right': 77.0},
  'right': {'node': {'feature': 'a', 'split': 21, 'cost': 3406.5},
   'left': 30.0,
   'right': 44.5}},
 {'node': {'feature': 'c', 'split': 8, 'cost': 968.0952380952381},
  'left': {'node': {'feature': 'a', 'split': 44, 'cost': 332.0},
   'left': 61.0,
   'right': 43.0},
  'right': 12.666666666666666},
 {'node': {'feature': 'a', 'split': 39, 'cost': 2696.25},
  'left': {'node': {'feature': 'a', 'split': 9, 'cost': 2347.5},
   'left': 40.5,
   'right': 52.5},
  'right': {'node': {'feature': 'a', 'split': 47, 'cost': 96.0},
   'left': 20.0,
   'right': 29.0}}]

Here we have a forest of 3 trees with a max depth of 2 and a minimum node size of 3, based on `toy_data`.
## Making it random
The final building block is making the trees random forest trees. Recall that this means for each node we limited the available predictors to a random sample of the predictor set. And we have actually done this in our `get_split_costs` function earlier. Let's take a closer look.

In [43]:
def get_split_costs(df):
    
    '''calulate the cost for each split'''
    
    return flatten_list(
        [[cost_function(split) for split in get_feature_splits(
            df.iloc[:,-1], df.iloc[:,:-1].loc[:, feature])] for feature in rand.sample(list(
            df.iloc[:,:-1].columns), int(math.sqrt(len(list(df.iloc[:,:-1].columns)))))])

The last two lines of this function limit the predictors to be assesed to be those randomly selected from all predictors. The size of the sample is the square root of the total number of predictors.  The feature sample size in generals should be treated as a hyperparameter to be tuned. I have chosen the sqrt(p) here for convenience.

And just like that you have a random forest. Let's put it all together.

## Growing a random forest

### The cost function

In [44]:
def combined_RSS(y1, y2):
    
    '''calculate the combined residual sum of squares for y1 and y1'''
    
    return np.sum((y1 - np.mean(y1))**2) + np.sum((y2 - np.mean(y2))**2)

### Selecting a node

In [ ]:
import pandas as pd
import numpy as np
import math
import random as rand


def get_feature_splits(y, f):
   
    '''split the y variable by split s in feature f'''
    
    return ([{'feature' : f.name, 'split' :  s, 'left': y[f < s],
              'right' :  y[f >= s]} for s in f])


def cost_function(feature_split): 
    
    '''calculate the total RSS for the splits defined by splitting feature f at split s'''
    
    return ({'feature' : feature_split['feature'],
              'split' : feature_split['split'],
              'cost' : combined_RSS(feature_split['left'], feature_split['right'])
            }
           )


def flatten_list(l):
    return [item for sublist in l for item in sublist]


def get_split_costs(df):
    
    '''calulate the cost for each split'''
    
    return flatten_list(
        [[cost_function(split) for split in get_feature_splits(
            df.iloc[:,-1], df.iloc[:,:-1].loc[:, feature])] for feature in rand.sample(list(
            df.iloc[:,:-1].columns), int(math.sqrt(len(list(df.iloc[:,:-1].columns)))))])


def select_node(split_costs): 
       
    '''select the lowest cost node'''
    
    return (rand.choice([{'feature' : split['feature'], 'split' : split['split'],
                          'cost' : split['cost']}
                         for split in split_costs if split['cost'] == np.min(
                             [split['cost'] for split in split_costs])]))
    

### Growing a tree


In [ ]:
def terminal_node(node_y):
    
    '''return the prediction for the terminal node'''
    
    return np.mean(node_y)


def grow_tree(df, node, depth, max_depth = 1, min_size = 5 ):
    
    ''' recursively grow a decision tree by applying the function to each node it 
    returns until max depth of min size criteria is met '''
    
    left = df.loc[df.loc[:, node['feature']] < node['split']]
    right = df.loc[df.loc[:, node['feature']] >= node['split']]
    
    if left.empty or right.empty:
        return terminal_node(list(left.iloc[:, -1]) + list(right.iloc[:, -1]))
        
    elif depth >= max_depth:
        return {'node': node, 
                'left': terminal_node(left.iloc[:, -1]), 
                'right': terminal_node(right.iloc[:, -1])}
    
    else:
        return {'node' : node,
                
                'left' : (lambda x: terminal_node(list(x.iloc[:, -1])) 
                          if len(x.iloc[:, -1]) <= min_size
                          else grow_tree(x, select_node(get_split_costs(x)),
                                        depth + 1, max_depth, min_size))(left),
                
                'right' : (lambda x: terminal_node(list(x.iloc[:, -1])) 
                           if len(x.iloc[:, -1]) <= min_size 
                           else grow_tree(x, select_node(get_split_costs(x)),
                                        depth + 1, max_depth, min_size))(right)
               }
    
    

## Growing a forest.

In [46]:
def bootstrap(df, random_state):
    
    ''' return a random sample of the same size as the df, with replacement'''
    
    return df.sample(len(df), replace = True, random_state = random_state)


def grow_random_forest(df, max_depth = 1, min_size = 5, no_of_trees = 10):  
    
    ''' return a specified number of random forest decision trees, 
    each based on a bootstrapped sample of df '''
    
    return [grow_tree(
        bootstrap(df, i), select_node( get_split_costs(bootstrap(df, i))),
        1, max_depth, min_size) for i in range(0, no_of_trees)]


## Wrap up

So that is one way to implement the random forest algorithm.  For more intel on the theory see:  [Introduction to Stastical Learning](https://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf) and [Elements of Statistical Learning](https://web.stanford.edu/~hastie/Papers/ESLII.pdf). 

For completeness I've included some code to make predictions and some output performance metrics and an example with the sklearn boston dataset.

In [47]:
def tree_predict_row(row, tree):
    
    '''use the trained tree to partition the feature space of the test observation and return 
    the partition prediction value, which for regression is the mean of the partition'''
    
    if row[tree['node']['feature']] <  tree['node']['split']:
        if not isinstance(tree['left'], dict):
            return tree['left']
        else:
            return tree_predict_row(row, tree['left'])
    else:
         if not isinstance(tree['right'], dict):
            return tree['right']
         else:
            return tree_predict_row(row, tree['right'])
        
       



In [48]:
def forest_predict_row(row, forest):
    
    ''' predicts each row of df for each tree than takes the average across 
    all trees for each row '''
    
    return np.mean([tree_predict_row(row, tree) for tree in forest])

In [49]:
def predict(df, forest):

    ''' applies predict row function to a df of test observations '''
    
    return df.apply(forest_predict_row, axis = 1, forest = forest)

In [50]:
def mse(y, predicted_y):
    
    ''' returns the mean square error: the sum of squyared differences between predicted y and actual y'''
    
    return np.sum((y - predicted_y)**2)

In [51]:
def total_sum_of_squares(y):

    '''returns the total sum of squares: the sum of squared difference between y and the mean of y i.e var(y)'''
    
    return np.sum((y - np.mean(y))**2)

In [52]:
def r_squared(y, predicted_y):
   
    ''' retruns to proportion of variance attributable the model: 1 - the ratio of mse to total sum of squares.'''

    return 1 - (mse(y, predicted_y)/total_sum_of_squares(y))

In [54]:
from sklearn import datasets
# training data
y = datasets.load_boston()['target']
X = datasets.load_boston()['data']
columns = datasets.load_boston()['feature_names']
data = pd.DataFrame(X)
data.columns = columns
data.loc[:, 'y'] = y

train = data.sample(frac =0.7)
test = data[~data.isin(train)].dropna()

In [55]:
boston_forest = grow_random_forest(train, max_depth = 5, min_size = 5, no_of_trees = 5)

In [56]:
display(boston_forest)

[{'node': {'feature': 'LSTAT', 'split': 9.74, 'cost': 14469.636662539735},
  'left': {'node': {'feature': 'LSTAT',
    'split': 4.67,
    'cost': 5883.217384219553},
   'left': {'node': {'feature': 'AGE',
     'split': 49.7,
     'cost': 1882.7201503759397},
    'left': {'node': {'feature': 'DIS',
      'split': 7.3967,
      'cost': 926.0085714285715},
     'left': {'node': {'feature': 'ZN',
       'split': 80.0,
       'cost': 188.50000000000003},
      'left': 34.49999999999999,
      'right': 50.0},
     'right': 30.0},
    'right': {'node': {'feature': 'RM',
      'split': 6.943,
      'cost': 186.7923076923076},
     'left': 22.8,
     'right': {'node': {'feature': 'B',
       'split': 385.05,
       'cost': 135.09999999999997},
      'left': 47.6,
      'right': 43.6}}},
   'right': {'node': {'feature': 'CRIM',
     'split': 9.2323,
     'cost': 3026.8851428571434},
    'left': {'node': {'feature': 'RM',
      'split': 6.842,
      'cost': 1452.8326352067868},
     'left': {'nod

In [57]:
boston_predictions = predict(test, boston_forest)

In [58]:
display(mse(test.iloc[:, -1], boston_predictions))
display(r_squared(test.iloc[:, -1],boston_predictions))

3088.0329382836526

0.8029567917599614